# 데이터 로딩 및 확인

In [15]:
import glob, os   # golb: 디렉토리 내 파일 찾는 거
import pandas as pd

all_files = glob.glob(os.path.join(r'.\dataset\topics', '*.data'))
print(type(all_files))
print(all_files[:2])

df = pd.read_table(all_files[0], encoding='latin1')
print(df.shape)
print(df.head())
print(df.to_string())

<class 'list'>
['.\\dataset\\topics\\transmission_toyota_camry_2007.txt.data', '.\\dataset\\topics\\voice_garmin_nuvi_255W_gps.txt.data']
(154, 1)
   After slowing down, transmission has to be  kicked  to speed up .
0   I wonder if the people who have problems with...                
1   The transmission, I don't know what to tell y...                
2  I did not notice any hesitation with the trans...                
3   I've had the same transmission problems hesit...                
4   I immediately starting noticing transmission ...                
                                                                                                                                                                                                                                                                                                                                                                             After slowing down, transmission has to be  kicked  to speed up .
0       

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [22]:
path = r'./dataset/topics/'
all_files = glob.glob(os.path.join(path, '*.data'))   # 파일명들 리스트로 가져옴
file_names = []
opinion_text = []
for file in all_files:
    df = pd.read_table(file, encoding='latin1')
    filename = file.split('\\')[-1]  # 경로 구분자 중 가장 뒤의 파일명만 저장
    filename = filename.split('.')[0]
    file_names.append(filename)
    opinion_text.append(df.to_string())  # df 값을 문자열로 변환

document_df = pd.DataFrame({'filename':file_names, 'opinion_text':opinion_text})
print(document_df.shape)
display(document_df.head())

(51, 2)


filename  \
0  transmission_toyota_camry_2007   
1      voice_garmin_nuvi_255W_gps   
2  bathroom_bestwestern_hotel_sfo   
3    updates_garmin_nuvi_255W_gps   
4             video_ipod_nano_8gb   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

# TfidVectorizer의 tokenizer 인자로 사용될 어근 변환 함수 선언

In [26]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [30]:
print(ord('A'))
print(chr(65))

65
A


- translate 함수
>- 문자열 내에 특정 문자를 다른 문자로 일괄 변경하는 함수
>- 매개 변수로 변경하고자 하는 정보를 딕셔너리로 전달한다. {원본 문자 유니코드:변경 문자 유니코드} 

In [34]:
# maketrans(): 원래 문자와 바꿀 문자에 대한 맵핑 정보(딕셔너리) 생성
trans = str.maketrans('[],','   ')
print(trans)
text = '[고양이, 강아지, 다람쥐, 토끼]'
text = text.translate(trans)
print(text)

{91: 32, 93: 32, 44: 32}
 고양이  강아지  다람쥐  토끼 


In [35]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# ord(): 매개 변수로 전달된 문자에 해당하는 유니코드를 반환 <-> chr(): 
remove_punct_dict = {ord(punct):None for punct in string.punctuation} # 구두점 하나씩을 가져와 유니코드로 변환
lemmar = WordNetLemmatizer()

# 입력된 문장을 받아 -> 소문자 변환 -> 구두점 제거 -> 단어 토큰화 -> 어근 변환
def LemNormalize(text):
    tokens = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
    return [lemmar.lemmatize(token) for token in tokens]

# TF-IDF 기반 피처 벡터화
- min_df: 전체 문서에 걸쳐 너무 낮은 빈도를 갖는 단어 피처를 제외
>- 정수: 지정된 개수 이하로 나타나는 단어는 피처 추출에서 제외
>- 실수(0.0~1.0): 지정한 백분율 이하 빈도수까지의 단어는 피처 추출에서 제외 (ex. 0.05 -> 하위 5% 이하의 빈도수를 가지는 단어는 피처 추출에서 제외)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

stopwords = nltk.corpus.stopwords.words('english')
tfidf_vext = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords, ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature_vect = tfidf_vext.fit_transform(document_df['opinion_text'])

C:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


In [38]:
print(feature_vect.shape)
print(feature_vect[0].toarray())

(51, 5014)
[[0. 0. 0. ... 0. 0. 0.]]


# 군집분석

## 군집 수 5개로 군집

In [40]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, random_state=0, max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_ceter = km_cluster.cluster_centers_

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [41]:
document_df['cluster_label'] = cluster_label
display(document_df.head())

filename  \
0  transmission_toyota_camry_2007   
1      voice_garmin_nuvi_255W_gps   
2  bathroom_bestwestern_hotel_sfo   
3    updates_garmin_nuvi_255W_gps   
4             video_ipod_nano_8gb   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [46]:
display(document_df[document_df['cluster_label'] == 0]['filename'])

2     bathroom_bestwestern_hotel_sfo
34       rooms_bestwestern_hotel_sfo
35           room_holiday_inn_london
36           rooms_swissotel_chicago
Name: filename, dtype: object

In [47]:
display(document_df[document_df['cluster_label'] == 1]['filename'])

1          voice_garmin_nuvi_255W_gps
3        updates_garmin_nuvi_255W_gps
4                 video_ipod_nano_8gb
5       accuracy_garmin_nuvi_255W_gps
9               buttons_amazon_kindle
12    directions_garmin_nuvi_255W_gps
13       display_garmin_nuvi_255W_gps
14      eyesight-issues_amazon_kindle
15                  features_windows7
16                fonts_amazon_kindle
23            keyboard_netbook_1005ha
27           navigation_amazon_kindle
31                price_amazon_kindle
37     satellite_garmin_nuvi_255W_gps
38        screen_garmin_nuvi_255W_gps
39               screen_ipod_nano_8gb
41              screen_netbook_1005ha
45           size_asus_netbook_1005ha
46                sound_ipod_nano_8gb
47         speed_garmin_nuvi_255W_gps
Name: filename, dtype: object

In [48]:
display(document_df[document_df['cluster_label'] == 2]['filename'])

0     transmission_toyota_camry_2007
6         battery-life_amazon_kindle
7         battery-life_ipod_nano_8gb
8        battery-life_netbook_1005ha
29     performance_honda_accord_2008
30        performance_netbook_1005ha
48                    speed_windows7
Name: filename, dtype: object

In [49]:
display(document_df[document_df['cluster_label'] == 3]['filename'])

10        comfort_honda_accord_2008
11        comfort_toyota_camry_2007
20    gas_mileage_toyota_camry_2007
21       interior_honda_accord_2008
22       interior_toyota_camry_2007
26        mileage_honda_accord_2008
32        quality_toyota_camry_2007
40          seats_honda_accord_2008
Name: filename, dtype: object

In [50]:
display(document_df[document_df['cluster_label'] == 4]['filename'])

17            food_holiday_inn_london
18             food_swissotel_chicago
19         free_bestwestern_hotel_sfo
24     location_bestwestern_hotel_sfo
25        location_holiday_inn_london
28      parking_bestwestern_hotel_sfo
33           price_holiday_inn_london
42      service_bestwestern_hotel_sfo
43         service_holiday_inn_london
44    service_swissotel_hotel_chicago
49        staff_bestwestern_hotel_sfo
50            staff_swissotel_chicago
Name: filename, dtype: object

## 군집 수 3개로 군집

In [51]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, random_state=0, max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_ceter = km_cluster.cluster_centers_

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [52]:
document_df['cluster_label'] = cluster_label

In [53]:
display(document_df[document_df['cluster_label'] == 0]['filename'])

2      bathroom_bestwestern_hotel_sfo
17            food_holiday_inn_london
18             food_swissotel_chicago
19         free_bestwestern_hotel_sfo
24     location_bestwestern_hotel_sfo
25        location_holiday_inn_london
28      parking_bestwestern_hotel_sfo
33           price_holiday_inn_london
34        rooms_bestwestern_hotel_sfo
35            room_holiday_inn_london
36            rooms_swissotel_chicago
42      service_bestwestern_hotel_sfo
43         service_holiday_inn_london
44    service_swissotel_hotel_chicago
49        staff_bestwestern_hotel_sfo
50            staff_swissotel_chicago
Name: filename, dtype: object

In [54]:
display(document_df[document_df['cluster_label'] == 1]['filename'])

1          voice_garmin_nuvi_255W_gps
3        updates_garmin_nuvi_255W_gps
4                 video_ipod_nano_8gb
5       accuracy_garmin_nuvi_255W_gps
6          battery-life_amazon_kindle
7          battery-life_ipod_nano_8gb
8         battery-life_netbook_1005ha
9               buttons_amazon_kindle
12    directions_garmin_nuvi_255W_gps
13       display_garmin_nuvi_255W_gps
14      eyesight-issues_amazon_kindle
15                  features_windows7
16                fonts_amazon_kindle
23            keyboard_netbook_1005ha
27           navigation_amazon_kindle
30         performance_netbook_1005ha
31                price_amazon_kindle
37     satellite_garmin_nuvi_255W_gps
38        screen_garmin_nuvi_255W_gps
39               screen_ipod_nano_8gb
41              screen_netbook_1005ha
45           size_asus_netbook_1005ha
46                sound_ipod_nano_8gb
47         speed_garmin_nuvi_255W_gps
48                     speed_windows7
Name: filename, dtype: object

In [55]:
display(document_df[document_df['cluster_label'] == 2]['filename'])

0     transmission_toyota_camry_2007
10         comfort_honda_accord_2008
11         comfort_toyota_camry_2007
20     gas_mileage_toyota_camry_2007
21        interior_honda_accord_2008
22        interior_toyota_camry_2007
26         mileage_honda_accord_2008
29     performance_honda_accord_2008
32         quality_toyota_camry_2007
40           seats_honda_accord_2008
Name: filename, dtype: object

# 군집별 핵심 단어 추출하기
- cluster_centers_
>- 단어 피처가 군집의 중심으로부터 얼마나 가깝게 있는지를 군집 내의 피처들과의 상대 위치로 나타낸 값
>- 0-1 사이의 값으로 1에 가까울 수록 중심과 가까운 것을 의미

In [58]:
cluster_centers = km_cluster.cluster_centers_
# (3, 5014)
# 3: 군집수
# 5014: 피처(단어)의 수
print(cluster_centers.shape)
print(cluster_centers[0][:10])

(3, 5014)
[0.         0.00173941 0.00413856 0.         0.00016861 0.
 0.         0.00232673 0.00071782 0.00235026]


## 군집별 상위 n개의 핵심 단어 추출 함수 정의 
- 매개 변수 설명
>- cluster_model: KMeans 모델 객체
>- cluster_data: 리뷰 문서 및 군집 결과 값을 가지고 있는 DataFrame 객체
>- feature_names: 피처 카운터 벡터화 된 각 피처(단어)의 이름
>- cluster_num: 군집 수
>- top_n_features: 군집별 확인하고자 하는 상위 n개의 피처(단어) 수

In [59]:
def get_cluster_details(cluster_model, cluster_data, feature_names, cluster_num, top_n_features=10):
    cluster_details = {}
    centroid_feature_ordered_idx = cluster_model.cluster_centers_.argsort()[:,::-1]

    for num in range(cluster_num):   # 군집 수만큼 반복 수행
        # {0:{}}
        cluster_details[num] = {}  # 개별 군집 정보를 담을 사전 객체 초기화 
        # {0:{'cluster':0}} 
        cluster_details[num]['cluster'] = num
        top_feature_indexes = centroid_feature_ordered_idx[num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        # 군집의 중심으로부터 가까운 상위 n개 단어의 중심점으로부터의 거리 값 목록
        top_features_values = cluster_model.cluster_centers_[num, top_feature_indexes].tolist()  # numpy 다차원 배열 -> 리스트

        cluster_details[num]['top_features'] = top_features
        cluster_details[num]['top_features_value'] = top_features_values
        filenames = cluster_data[cluster_data['cluster_label'] == num]['filename']
        filenames = filenames.values.tolist()  # 시리즈 객체.values -> np 다차원 배열 -> 리스트
        cluster_details[num]['filenames'] = filenames

    return cluster_details


In [60]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print(f'#### Cluster: {cluster_num} ####')
        print(f'Top Features: {cluster_detail["top_features"]}')
        print(f'Review 파일명: {cluster_detail["filenames"][:5]}')
        print('='*30)

In [61]:
feature_names = tfidf_vext.get_feature_names_out()
cluster_details = get_cluster_details(km_cluster, document_df, feature_names, 3)
print(print_cluster_details(cluster_details))

#### Cluster: 0 ####
Top Features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Review 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo']
#### Cluster: 1 ####
Top Features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'video', 'direction', 'feature', 'voice']
Review 파일명: ['voice_garmin_nuvi_255W_gps', 'updates_garmin_nuvi_255W_gps', 'video_ipod_nano_8gb', 'accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle']
#### Cluster: 2 ####
Top Features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Review 파일명: ['transmission_toyota_camry_2007', 'comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008']
None
